In [2]:
from transformers import BertTokenizer, BertForQuestionAnswering

In [3]:
import torch
import warnings
warnings.filterwarnings("ignore")

In [4]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

In [6]:
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def predict_answer(question, context):
    encoding = tokenizer.encode_plus(question, context, return_tensors = "pt", max_length = 512, truncation = True)

    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]

    with torch.inference_mode():
        start_scores, end_scores = model(input_ids, attention_mask = attention_mask, return_dict = False)

    start_index = torch.argmax(start_scores, dim = 1).item()
    end_index = torch.argmax(end_scores, dim = 1).item()

    answer_tokens = tokenizer.convert_ids_to_tokens(input_ids[0][start_index: end_index+1])
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    return answer

In [11]:
question = "What is the capital of Turkiye"
context = "Turkiye, officially Turkish Republic, is a country whose capital is Ankara."

result = predict_answer(question, context)
print(f"The Result: {result.upper()}")

The Result: ANKARA
